問題設定:探討「不同的行銷活動」與「顧客回購次數多寡」與｢利潤」的關係

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime, tzinfo, timezone
import plotly.express as px
import plotly as py
import os

In [2]:
#fec=pd.read_csv('*.csv',index_col=None,nrows=10)
all_data = pd.read_csv('sales_data.csv',index_col=None)
all_data

,Unnamed: 0,顏色,單價,成本,系列,產品,訂單時間,會員,性別,年紀,廣告代號all,尺寸
0,0,NaN,391,240,系列4,產品4-1,2016-01-01T03:19:35,10052013,FEMALE,32.0,廣告_YND_pid,無
1,1,NaN,238,137,系列4,產品4-2,2016-01-01T03:19:35,10052013,FEMALE,32.0,廣告_YND_pid,無
2,2,watermelonred,434,253,系列4,產品4-3,2016-01-01T03:19:35,10052013,FEMALE,32.0,廣告_YND_pid,S
3,3,NaN,339,205,系列4,產品4-4,2016-01-01T03:19:35,10052013,FEMALE,32.0,廣告_YND_pid,無
4,4,white,382,223,系列4,產品4-3,2016-01-01T03:19:35,10052013,FEMALE,32.0,廣告_YND_pid,S
...,...,...,...,...,...,...,...,...,...,...,...,...
353219,353219,NaN,350,245,系列207,產品207-35,2019-11-12T14:44:14,10176032,FEMALE,18.0,廣告_自然流量,無
353220,353220,NaN,350,245,系列207,產品207-35,2019-11-13T09:37:06,10225036,FEMALE,24.0,廣告_自然流量,無
353221,353221,NaN,350,245,系列207,產品207-35,2019-11-14T18:34:45,10225049,NaN,28.0,廣告_KBDG_MK,無
353222,353222,NaN,350,245,系列207,產品207-35,2019-11-14T18:34:45,10225049,NaN,28.0,廣告_KBDG_MK,無


分析客戶透過甚麼「行銷活動」的吸引，而「回購產品」的情形與「利潤」之間的關係

1.廣告代號all: 每筆訂單不同時期的行銷活動

2.會員:每一位會員的專屬編號

3.訂單時間:會員送出訂單的時間

4.系列:由許多類似產品所組成的系列(產品1-1、產品1-2、產品1-3組成系列1)

5.單價:每一個產品的單價

6.成本:每一個產品的成本

7.利潤:由同一個產品的單價減掉它的成本所得出的金額

# 資料清理

In [3]:
# 從資料中取得所需項目
all_data = all_data[['廣告代號all','會員','訂單時間','系列','單價','成本']]
# 計算出利潤項目(單價-成本
all_data['利潤'] = all_data['單價'] - all_data['成本']
all_data

<ipython-input-3-6419694574ba>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data['利潤'] = all_data['單價'] - all_data['成本']


,廣告代號all,會員,訂單時間,系列,單價,成本,利潤
0,廣告_YND_pid,10052013,2016-01-01T03:19:35,系列4,391,240,151
1,廣告_YND_pid,10052013,2016-01-01T03:19:35,系列4,238,137,101
2,廣告_YND_pid,10052013,2016-01-01T03:19:35,系列4,434,253,181
3,廣告_YND_pid,10052013,2016-01-01T03:19:35,系列4,339,205,134
4,廣告_YND_pid,10052013,2016-01-01T03:19:35,系列4,382,223,159
...,...,...,...,...,...,...,...
353219,廣告_自然流量,10176032,2019-11-12T14:44:14,系列207,350,245,105
353220,廣告_自然流量,10225036,2019-11-13T09:37:06,系列207,350,245,105
353221,廣告_KBDG_MK,10225049,2019-11-14T18:34:45,系列207,350,245,105
353222,廣告_KBDG_MK,10225049,2019-11-14T18:34:45,系列207,350,245,105


In [4]:
# 將 [廣告代號all] --> [廣告代號]
all_data = all_data.rename(columns={'廣告代號all':'廣告代號',})

# 將廣告代號中的項目「廣告_」去除
all_data['廣告代號'] = all_data['廣告代號'].str.replace('廣告_','')

# 把子項廣告歸納回大廣告
all_data['廣告代號'] = all_data['廣告代號'].str.split('_').str.get(0)
all_data

,廣告代號,會員,訂單時間,系列,單價,成本,利潤
0,YND,10052013,2016-01-01T03:19:35,系列4,391,240,151
1,YND,10052013,2016-01-01T03:19:35,系列4,238,137,101
2,YND,10052013,2016-01-01T03:19:35,系列4,434,253,181
3,YND,10052013,2016-01-01T03:19:35,系列4,339,205,134
4,YND,10052013,2016-01-01T03:19:35,系列4,382,223,159
...,...,...,...,...,...,...,...
353219,自然流量,10176032,2019-11-12T14:44:14,系列207,350,245,105
353220,自然流量,10225036,2019-11-13T09:37:06,系列207,350,245,105
353221,KBDG,10225049,2019-11-14T18:34:45,系列207,350,245,105
353222,KBDG,10225049,2019-11-14T18:34:45,系列207,350,245,105


擷取分析資料 「2018年系列1」的銷售資料 

In [5]:
# 產品分群
product_data = all_data.groupby('系列')


# 存取系列1
product_1=pd.DataFrame(product_data.get_group('系列1'))

#刪除之後不需要的欄位
del product_1['單價']
del product_1['成本']

product_1

,廣告代號,會員,訂單時間,系列,利潤
74181,critei,11734,2016-01-02T14:16:32,系列1,99
74182,critei,11734,2016-01-02T14:16:32,系列1,103
74183,自然流量,10012645,2016-01-03T12:25:18,系列1,181
74184,自然流量,10012645,2016-01-03T12:25:18,系列1,192
74185,自然流量,10012645,2016-01-03T12:25:18,系列1,268
...,...,...,...,...,...
103313,自然流量,10225463,2019-11-18T18:14:41,系列1,240
103314,自然流量,10225463,2019-11-18T18:14:41,系列1,120
103315,自然流量,10225463,2019-11-18T18:16:52,系列1,240
103316,自然流量,10225463,2019-11-18T18:16:52,系列1,120


訂單時間格式轉換成可計算時間形式(datetime)

In [6]:
# 把訂單時間中的"T"去掉
product_1['訂單時間']=product_1['訂單時間'].str.replace('T',' ')
product_1[:5]


,廣告代號,會員,訂單時間,系列,利潤
74181,critei,11734,2016-01-02 14:16:32,系列1,99
74182,critei,11734,2016-01-02 14:16:32,系列1,103
74183,自然流量,10012645,2016-01-03 12:25:18,系列1,181
74184,自然流量,10012645,2016-01-03 12:25:18,系列1,192
74185,自然流量,10012645,2016-01-03 12:25:18,系列1,268


In [7]:
# 將訂單時間改成datatime形式
product_1['訂單時間'] = pd.to_datetime(product_1['訂單時間'])
product_1[:5]

# 檢查「訂單時間」是成功更改為datetime
product_1.dtypes

廣告代號            object
會員               int64
訂單時間    datetime64[ns]
系列              object
利潤               int64
dtype: object

In [8]:
# 找到系列1中 所有的廣告名稱
ad_pd1 = product_1['廣告代號'].unique().tolist()
ad_pd1

['critei',
 '自然流量',
 'B2K',
 'ikip',
 'qdwit',
 'qpw',
 'KDPOD',
 'B2KPOD',
 'KDMK',
 'cridgewell',
 'B2KMK',
 'KDP',
 'B2KP',
 'edmM',
 'edmP',
 'KDPM',
 'KDM',
 'GINEP',
 'pic',
 'B2KM',
 'edmD',
 'edm',
 'KDMP',
 'edmMK',
 'B2KMP',
 'GqmerP',
 'GINEMP',
 'edmMP',
 'GINEMK',
 'B2KDG',
 'GqmerMP',
 'KBP',
 'KBMP',
 'KBMK',
 'IGMP',
 'KBDG',
 'IGDG',
 'DPPP',
 'exhicitiicP',
 'Ddwirds',
 'DMDMP']

取出分析資料(系列1-2018年度)

In [9]:
# 先設立一個可以放2018年每個月資料的list
original_2018 = []
for i in range(1,13):
    # 當月份等於12月時，執行下方程式
    if i == 12:
        # 設立選取資料起始時間 (2018年 12月 01日)
        begin = datetime(2018, i, 1)
        
        # 設立選取資料結束時間 (2019年 01月 01日)
        end = datetime(2019, i, 1)
        
        # 選取資料 (結束時間之前)
        pd1_2018 = product_1[product_1['訂單時間'] <= end]
        
        # 選取資料 (開始時間之後)
        pd1_2018 = pd1_2018[pd1_2018['訂單時間'] >= begin]
        
        # 將12月的資料存放每個月資料的list中
        original_2018.append(pd1_2018)
        # 當月份不等於12月時，執行下方程式
    else:
        # 設立選取資料起始時間 (2018年 **月 01日)
        begin = datetime(2018, i, 1)
        
        # 設立選取資料結束時間(2018年 **月+1個月 01日) 01/01 ~ 02/01
        end = datetime(2018, i+1, 1)
        
        # 選取資料 (結束時間之前)
        pd1_2018 = product_1[product_1['訂單時間'] <= end]
        
        # 選許資料 (開始時間之後)
        pd1_2018 = pd1_2018[pd1_2018['訂單時間'] >= begin]
        
        # 將該月份的資料存放到每個月資料的list中
        original_2018.append(pd1_2018)

In [10]:
# 廣告月份訂單資料
original_2018

[         廣告代號        會員                訂單時間   系列   利潤
 85965    自然流量  10135347 2018-01-01 01:59:29  系列1  478
 85966    自然流量  10135347 2018-01-01 01:59:29  系列1  478
 85967     KDP  10135248 2018-01-01 03:01:44  系列1  244
 85968     KDP  10135248 2018-01-01 03:01:44  系列1  254
 85969     KDP     23524 2018-01-01 07:14:49  系列1  285
 ...       ...       ...                 ...  ...  ...
 86558    edmP     69118 2018-01-31 17:57:13  系列1  319
 86559    自然流量  10093209 2018-01-31 18:18:27  系列1  469
 86560  critei  10076597 2018-01-31 23:16:53  系列1  544
 86561    自然流量  10137781 2018-01-31 23:56:30  系列1  429
 86562    自然流量  10137781 2018-01-31 23:56:30  系列1  319
 
 [598 rows x 5 columns],
          廣告代號        會員                訂單時間   系列   利潤
 86563    B2KP  10078662 2018-02-01 00:03:00  系列1  334
 86564    自然流量  10087441 2018-02-01 01:22:23  系列1  380
 86565    B2KP  10138414 2018-02-01 03:10:52  系列1  275
 86566    自然流量  10137874 2018-02-01 03:13:53  系列1  164
 86567    自然流量  10136746 2018-02-01 04

# 建立會員消費紀錄資料庫
分類好每個月的訂單資料後，從每個月的每筆資料分析每一個會員分別在「哪個月」購買系列1的產品，並且判斷出他是「第幾次購買」，是因為「甚麼廣告」吸引會員購買，將上述資訊使用二維字典的方式建立成一個資料庫

In [11]:
# 建立每個會員的消費紀錄
customer_record = {}
for mon in range(len(original_2018)):
    #print(original_2018[mon])
    
    # 將每個訂單的利潤計算出來存成list
    profit_gp = original_2018[mon].groupby('訂單時間')['利潤'].sum().tolist()
    #print(profit_gp)
    
    # 把訂單時間重複的drop掉 同一張發票的消費紀錄合併再一起
    original_2018[mon] = original_2018[mon].drop_duplicates(subset=['訂單時間'])
    
    # 把利潤補回原本的df
    original_2018[mon]['利潤'] = profit_gp
    #print(original_2018[mon])
    
    # index重新排序
    original_2018[mon]= original_2018[mon].reset_index(drop = True)
    # print(original_2018)
    
    a =original_2018[mon]
    
    for i in range(len(a['會員'])):
        
            # 如果會員 不在 字典的Keys中，執行下方程式
            if str(a['會員'][i]) not in customer_record.keys():
                
                # 新增會員到keys中，並放上對應的消費紀錄
                customer_record[str(a['會員'][i]) ] ={'1':str(mon+1)+'月_'+a['廣告代號'][i]} 
            # 如果會員已經在字典的Kesy中執行下方程式
            else:
                # 直接在原有的會員Kesy下，放上對應的消費紀錄
                customer_record[str(a['會員'][i])][str(len(customer_record[str(a['會員'][i] )])+1)]=str(mon+1)+'月_'+a['廣告代號'][i]

<ipython-input-11-1e516f5f1696>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_2018[mon]['利潤'] = profit_gp


In [12]:
customer_record
# {會員:{1(第一次購買):1月自然流量(1月的行銷活動消費)}}

{'10135347': {'1': '1月_自然流量', '2': '2月_自然流量'},
 '10135248': {'1': '1月_KDP'},
 '23524': {'1': '1月_KDP'},
 '10080582': {'1': '1月_自然流量',
  '2': '6月_自然流量',
  '3': '12月_KDMP',
  '4': '12月_自然流量'},
 '10125335': {'1': '1月_自然流量',
  '2': '4月_自然流量',
  '3': '4月_自然流量',
  '4': '12月_KDMP',
  '5': '12月_GINEP',
  '6': '12月_GINEP'},
 '10135378': {'1': '1月_critei'},
 '10114009': {'1': '1月_edmP'},
 '10134735': {'1': '1月_critei'},
 '10135392': {'1': '1月_自然流量'},
 '10109503': {'1': '1月_KDP'},
 '10135018': {'1': '1月_KDP'},
 '10135396': {'1': '1月_自然流量'},
 '10132365': {'1': '1月_自然流量', '2': '6月_KDP'},
 '10049260': {'1': '1月_critei', '2': '3月_critei'},
 '10130702': {'1': '1月_自然流量'},
 '10108847': {'1': '1月_KDP', '2': '5月_critei', '3': '7月_critei'},
 '62481': {'1': '1月_KDP'},
 '10134105': {'1': '1月_critei'},
 '10133968': {'1': '1月_ikip'},
 '10135429': {'1': '1月_自然流量'},
 '10133589': {'1': '1月_自然流量'},
 '10134441': {'1': '1月_自然流量'},
 '10135446': {'1': '1月_自然流量'},
 '10135444': {'1': '1月_自然流量'},
 '10086957': {'1': '1月_自

# 「會員消費紀錄資料庫」匯出「資料視覺化」(EDA)

彙整資料清單
從消費紀錄資料庫，比對時間後，建立2018年1 ~ 12月 的會員消費資料，並把每個月的資料彙整成一份清單

In [13]:
# 分類好每個月的資料(哪個會員在第幾個月因為甚麼廣告而購買，第幾次購買)
final_2018=[]

# 從1月開始蒐集
for mon in range(len(original_2018)):
    # 建立存取會員、廣告代號、第幾次購買之串列
    original_2018_id    = []
    original_2018_ad    = []
    original_2018_count = []
    
    # 美個會員編號都跑一次 i: 不同會員編號
    for i in customer_record.keys():
        
        # 查詢某會員在2018年購買系列1所有的消費紀錄
        for j in customer_record[i].values():
            
            # 消費紀錄中若開頭為蒐集該月份則執行下方程式
            # str.startswith(str,beg=0,end=len(string)) 檢查字串內有無符合，若符合回傳True
            if j.startswith(str(mon+1)+'月'):
                # 存取該會員編號
                original_2018_id.append(i)
                print('會員編號:',i)
                
                # 存取其廣告代號
                original_2018_ad.append(j)
                print('廣告代號:',j)
                
                # 存取會員在這筆消費紀錄中是第幾次購買的舉例:'1'
                original_2018_count.append(list (customer_record[i].keys()) 
                                           [list (customer_record[i].values()).index(j)])
                
                print('第幾次購買:',list (customer_record[i].keys())
                                    [list(customer_record[i].values()).index(j)])
    # 建立DataFrame
    original_2018_df = pd.DataFrame({'會員':original_2018_id,
                                     '廣告代號':original_2018_ad,
                                     '第幾次購買':original_2018_count})

    # 存放到分類12個月的串列之中
    final_2018.append(original_2018_df)

會員編號: 10135347
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10135248
廣告代號: 1月_KDP
第幾次購買: 1
會員編號: 23524
廣告代號: 1月_KDP
第幾次購買: 1
會員編號: 10080582
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10125335
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10135378
廣告代號: 1月_critei
第幾次購買: 1
會員編號: 10114009
廣告代號: 1月_edmP
第幾次購買: 1
會員編號: 10134735
廣告代號: 1月_critei
第幾次購買: 1
會員編號: 10135392
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10109503
廣告代號: 1月_KDP
第幾次購買: 1
會員編號: 10135018
廣告代號: 1月_KDP
第幾次購買: 1
會員編號: 10135396
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10132365
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10049260
廣告代號: 1月_critei
第幾次購買: 1
會員編號: 10130702
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10108847
廣告代號: 1月_KDP
第幾次購買: 1
會員編號: 62481
廣告代號: 1月_KDP
第幾次購買: 1
會員編號: 10134105
廣告代號: 1月_critei
第幾次購買: 1
會員編號: 10133968
廣告代號: 1月_ikip
第幾次購買: 1
會員編號: 10135429
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10133589
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10134441
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10135446
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10135444
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10086957
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10132711
廣告代號: 1月_KDP
第幾次購買: 1
會員編號: 10102492
廣告

廣告代號: 1月_B2KP
第幾次購買: 1
會員編號: 10137090
廣告代號: 1月_critei
第幾次購買: 1
會員編號: 10137093
廣告代號: 1月_B2KP
第幾次購買: 1
會員編號: 10129792
廣告代號: 1月_B2KP
第幾次購買: 1
會員編號: 10127157
廣告代號: 1月_B2KP
第幾次購買: 1
會員編號: 10090427
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10137107
廣告代號: 1月_B2KP
第幾次購買: 1
會員編號: 10137041
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10125309
廣告代號: 1月_B2KP
第幾次購買: 1
會員編號: 10137146
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10098528
廣告代號: 1月_B2KP
第幾次購買: 1
會員編號: 10137190
廣告代號: 1月_B2KP
第幾次購買: 1
會員編號: 10137208
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10137240
廣告代號: 1月_B2KP
第幾次購買: 1
會員編號: 10137247
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10120379
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10137286
廣告代號: 1月_B2KP
第幾次購買: 1
會員編號: 10101316
廣告代號: 1月_B2KP
第幾次購買: 1
會員編號: 10137298
廣告代號: 1月_critei
第幾次購買: 1
會員編號: 10137490
廣告代號: 1月_B2KP
第幾次購買: 1
會員編號: 10137460
廣告代號: 1月_B2KP
第幾次購買: 1
會員編號: 10137610
廣告代號: 1月_B2KP
第幾次購買: 1
會員編號: 10137584
廣告代號: 1月_B2KP
第幾次購買: 1
會員編號: 10137615
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 10137563
廣告代號: 1月_critei
第幾次購買: 1
會員編號: 10137637
廣告代號: 1月_自然流量
第幾次購買: 1
會員編號: 57332
廣告代號: 1月_

廣告代號: 2月_自然流量
第幾次購買: 1
會員編號: 10104705
廣告代號: 2月_自然流量
第幾次購買: 1
會員編號: 10139586
廣告代號: 2月_B2KP
第幾次購買: 1
會員編號: 10139588
廣告代號: 2月_B2KP
第幾次購買: 1
會員編號: 10139133
廣告代號: 2月_critei
第幾次購買: 1
會員編號: 10013091
廣告代號: 2月_KDP
第幾次購買: 1
會員編號: 10075198
廣告代號: 2月_自然流量
第幾次購買: 1
會員編號: 10139609
廣告代號: 2月_自然流量
第幾次購買: 1
會員編號: 10139622
廣告代號: 2月_B2KP
第幾次購買: 1
會員編號: 10112025
廣告代號: 2月_edmP
第幾次購買: 1
會員編號: 10138872
廣告代號: 2月_critei
第幾次購買: 1
會員編號: 10139633
廣告代號: 2月_自然流量
第幾次購買: 1
會員編號: 10101472
廣告代號: 2月_自然流量
第幾次購買: 1
會員編號: 10139664
廣告代號: 2月_自然流量
第幾次購買: 1
會員編號: 10075282
廣告代號: 2月_自然流量
第幾次購買: 1
會員編號: 10135297
廣告代號: 2月_自然流量
第幾次購買: 1
會員編號: 10074988
廣告代號: 2月_自然流量
第幾次購買: 1
會員編號: 10134097
廣告代號: 2月_自然流量
第幾次購買: 1
會員編號: 10131003
廣告代號: 2月_自然流量
第幾次購買: 1
會員編號: 10039639
廣告代號: 2月_自然流量
第幾次購買: 1
會員編號: 10139701
廣告代號: 2月_自然流量
第幾次購買: 1
會員編號: 10058394
廣告代號: 2月_edmP
第幾次購買: 1
會員編號: 10139771
廣告代號: 2月_自然流量
第幾次購買: 1
會員編號: 10139787
廣告代號: 2月_自然流量
第幾次購買: 1
會員編號: 10139790
廣告代號: 2月_自然流量
第幾次購買: 1
會員編號: 10074147
廣告代號: 2月_KDP
第幾次購買: 1
會員編號: 10139798
廣告代號: 2月_c

會員編號: 10141426
廣告代號: 3月_KDP
第幾次購買: 1
會員編號: 10096451
廣告代號: 3月_KDP
第幾次購買: 1
會員編號: 10083002
廣告代號: 3月_critei
第幾次購買: 1
會員編號: 29822
廣告代號: 3月_自然流量
第幾次購買: 1
會員編號: 10141451
廣告代號: 3月_自然流量
第幾次購買: 1
會員編號: 10141459
廣告代號: 3月_自然流量
第幾次購買: 1
會員編號: 10058313
廣告代號: 3月_GINEP
第幾次購買: 1
會員編號: 10141470
廣告代號: 3月_自然流量
第幾次購買: 1
會員編號: 10120909
廣告代號: 3月_KDP
第幾次購買: 1
會員編號: 10141083
廣告代號: 3月_自然流量
第幾次購買: 1
會員編號: 10141489
廣告代號: 3月_自然流量
第幾次購買: 1
會員編號: 10128548
廣告代號: 3月_自然流量
第幾次購買: 1
會員編號: 10008956
廣告代號: 3月_自然流量
第幾次購買: 1
會員編號: 10134211
廣告代號: 3月_自然流量
第幾次購買: 1
會員編號: 10141551
廣告代號: 3月_自然流量
第幾次購買: 1
會員編號: 10091360
廣告代號: 3月_KDP
第幾次購買: 1
會員編號: 10141574
廣告代號: 3月_critei
第幾次購買: 1
會員編號: 10141575
廣告代號: 3月_自然流量
第幾次購買: 1
會員編號: 10141535
廣告代號: 3月_自然流量
第幾次購買: 1
會員編號: 41277
廣告代號: 3月_KDP
第幾次購買: 1
會員編號: 10141584
廣告代號: 3月_自然流量
第幾次購買: 1
會員編號: 18223
廣告代號: 3月_edmP
第幾次購買: 1
會員編號: 10073539
廣告代號: 3月_KDPOD
第幾次購買: 1
會員編號: 10141280
廣告代號: 3月_自然流量
第幾次購買: 1
會員編號: 10042818
廣告代號: 3月_自然流量
第幾次購買: 1
會員編號: 10126154
廣告代號: 3月_自然流量
第幾次購買: 1
會員編號: 10113464
廣告代號:

廣告代號: 4月_KDP
第幾次購買: 1
會員編號: 10058605
廣告代號: 4月_KDP
第幾次購買: 1
會員編號: 10147673
廣告代號: 4月_自然流量
第幾次購買: 1
會員編號: 10123818
廣告代號: 4月_KDP
第幾次購買: 1
會員編號: 10147723
廣告代號: 4月_KDP
第幾次購買: 1
會員編號: 10147744
廣告代號: 4月_KDP
第幾次購買: 1
會員編號: 10147611
廣告代號: 4月_自然流量
第幾次購買: 1
會員編號: 10147794
廣告代號: 4月_自然流量
第幾次購買: 1
會員編號: 10147797
廣告代號: 4月_自然流量
第幾次購買: 1
會員編號: 10147801
廣告代號: 4月_B2KP
第幾次購買: 1
會員編號: 10147800
廣告代號: 4月_自然流量
第幾次購買: 1
會員編號: 10147803
廣告代號: 4月_自然流量
第幾次購買: 1
會員編號: 10099901
廣告代號: 4月_自然流量
第幾次購買: 1
會員編號: 10147813
廣告代號: 4月_KDP
第幾次購買: 1
會員編號: 10145910
廣告代號: 4月_自然流量
第幾次購買: 1
會員編號: 10145216
廣告代號: 4月_GINEP
第幾次購買: 1
會員編號: 10115564
廣告代號: 4月_自然流量
第幾次購買: 1
會員編號: 10147761
廣告代號: 4月_GINEP
第幾次購買: 1
會員編號: 10130360
廣告代號: 4月_GINEP
第幾次購買: 1
會員編號: 10147615
廣告代號: 4月_自然流量
第幾次購買: 1
會員編號: 10147893
廣告代號: 4月_KDP
第幾次購買: 1
會員編號: 10133001
廣告代號: 4月_KDP
第幾次購買: 1
會員編號: 10108847
廣告代號: 5月_critei
第幾次購買: 2
會員編號: 10115535
廣告代號: 5月_自然流量
第幾次購買: 3
會員編號: 10101518
廣告代號: 5月_critei
第幾次購買: 2
會員編號: 10138763
廣告代號: 5月_自然流量
第幾次購買: 3
會員編號: 10103666
廣告代號: 5月_自然流量

...

會員編號: 10195330
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10193801
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 1579
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10195357
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10195362
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10195366
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10195367
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10195373
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10167720
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10195374
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10195380
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10150529
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10072451
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10162685
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10195385
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10195390
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10195394
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10195383
廣告代號: 12月_GINEP
第幾次購買: 1
會員編號: 10195396
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10195401
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10195400
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10195404
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10195415
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10195399
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10195422
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 40241
廣告代號: 12月_KDMP
第

第幾次購買: 1
會員編號: 10196561
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10196321
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10196563
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10196195
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10194571
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10134110
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10196594
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10196597
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 37767
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10196323
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10189239
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10196611
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10196050
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10196629
廣告代號: 12月_GINEMK
第幾次購買: 1
會員編號: 10196629
廣告代號: 12月_GINEMK
第幾次購買: 1
會員編號: 10191588
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10196638
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10189669
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10196553
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10196654
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10196661
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10196664
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10181368
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10101214
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10101214
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10195866


會員編號: 10198711
廣告代號: 12月_B2KDG
第幾次購買: 1
會員編號: 10198717
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10198716
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10198721
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10198723
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10198722
廣告代號: 12月_B2KDG
第幾次購買: 1
會員編號: 10198725
廣告代號: 12月_GINEMP
第幾次購買: 1
會員編號: 10198731
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10198724
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10198718
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10198710
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10198727
廣告代號: 12月_B2KDG
第幾次購買: 1
會員編號: 10198733
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10198719
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 70143
廣告代號: 12月_GINEMP
第幾次購買: 1
會員編號: 10198736
廣告代號: 12月_GINEMP
第幾次購買: 1
會員編號: 10198741
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10119618
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10183766
廣告代號: 12月_B2KDG
第幾次購買: 1
會員編號: 10090903
廣告代號: 12月_B2KDG
第幾次購買: 1
會員編號: 10198744
廣告代號: 12月_B2KDG
第幾次購買: 1
會員編號: 10198747
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10198749
廣告代號: 12月_B2KDG
第幾次購買: 1
會員編號: 10126962
廣告代號: 12月_B2KDG
第幾次購買: 1
會員編號: 10115087
廣告代號: 12月_B2KDG
第幾次購買: 1
會員編號: 1019875

第幾次購買: 1
會員編號: 10199003
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 63422
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10125960
廣告代號: 12月_GINEMK
第幾次購買: 1
會員編號: 10040407
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10199523
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10199526
廣告代號: 12月_IGDG
第幾次購買: 1
會員編號: 10199524
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10199524
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10166853
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10198863
廣告代號: 12月_GINEMK
第幾次購買: 1
會員編號: 10199546
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10199546
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10199553
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10199554
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10199565
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10199565
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10199564
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10199569
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10199574
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10199581
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10199588
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10199605
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10199602
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10199620
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10198981
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10199627


第幾次購買: 1
會員編號: 10158999
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10223587
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10223564
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10169695
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10223606
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10223609
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10223601
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10199370
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10223630
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10223339
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10223280
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10124469
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10223684
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10223673
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10223713
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10223740
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10223746
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10188369
廣告代號: 12月_KBDG
第幾次購買: 1
會員編號: 10223755
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10223761
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10030349
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10222744
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10223827
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10223844
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10189238
廣告代號: 12月_自然流量
第幾次購買: 1
會員編號: 10223834
廣

In [14]:
final_2018

[           會員       廣告代號 第幾次購買
 0    10135347    1月_自然流量     1
 1    10135248     1月_KDP     1
 2       23524     1月_KDP     1
 3    10080582    1月_自然流量     1
 4    10125335    1月_自然流量     1
 ..        ...        ...   ...
 314     29996  1月_critei     1
 315     69118    1月_edmP     1
 316  10093209    1月_自然流量     1
 317  10076597  1月_critei     1
 318  10137781    1月_自然流量     1
 
 [319 rows x 3 columns],
            會員       廣告代號 第幾次購買
 0    10135347    2月_自然流量     2
 1    10134641    2月_自然流量     2
 2    10090740  2月_critei     2
 3    10136036    2月_自然流量     2
 4    10136093  2月_critei     2
 ..        ...        ...   ...
 311  10129017    2月_自然流量     1
 312  10140851     2月_KDP     1
 313  10140852    2月_自然流量     1
 314  10140862  2月_critei     1
 315  10124281   2月_GINEP     1
 
 [316 rows x 3 columns],
            會員       廣告代號 第幾次購買
 0    10049260  3月_critei     2
 1    10090740  3月_critei     3
 2    10136688    3月_自然流量     2
 3    10137563  3月_critei     3
 4    10137674   3

# 調整資料清單



將利潤欄位合併到會員資料內

In [15]:
original_2018

[       廣告代號        會員                訂單時間   系列    利潤
 0      自然流量  10135347 2018-01-01 01:59:29  系列1   956
 1       KDP  10135248 2018-01-01 03:01:44  系列1   498
 2       KDP     23524 2018-01-01 07:14:49  系列1   609
 3      自然流量  10080582 2018-01-01 07:19:18  系列1  1140
 4      自然流量  10125335 2018-01-01 08:59:57  系列1   512
 ..      ...       ...                 ...  ...   ...
 314  critei     29996 2018-01-31 17:43:54  系列1   564
 315    edmP     69118 2018-01-31 17:57:13  系列1   319
 316    自然流量  10093209 2018-01-31 18:18:27  系列1   469
 317  critei  10076597 2018-01-31 23:16:53  系列1   544
 318    自然流量  10137781 2018-01-31 23:56:30  系列1   748
 
 [319 rows x 5 columns],
        廣告代號        會員                訂單時間   系列   利潤
 0      B2KP  10078662 2018-02-01 00:03:00  系列1  334
 1      自然流量  10087441 2018-02-01 01:22:23  系列1  380
 2      B2KP  10138414 2018-02-01 03:10:52  系列1  275
 3      自然流量  10137874 2018-02-01 03:13:53  系列1  164
 4      自然流量  10136746 2018-02-01 04:21:51  系列1  389
 ..    

In [16]:
# 目的:為了跟上final_2018去做marge
# 補上月份
for i in range(len(original_2018)):
    original_2018[i]['廣告代號'] = str (i +1)+'月_'+original_2018[i]['廣告代號']
# 目的:使original_2018和 final_2018 的會員欄位內容「type」相同
# 將original_2018中的會員轉成string
for i in range(len(original_2018)):
    original_2018[i]['會員'] = original_2018[i]['會員'].astype(str)
    print(str(i+1)+'月check!')

1月check!
2月check!
3月check!
4月check!
5月check!
6月check!
7月check!
8月check!
9月check!
10月check!
11月check!
12月check!


In [17]:
for i in range(len(final_2018)):
    
    final_2018[i] = final_2018[i].merge(original_2018[i],how='inner',
                                        left_on=['會員','廣告代號'], 
                                       right_on = ['會員','廣告代號'])
    # 刪除重複merge到的錯誤
    # df.drop_duplicates(subset=None, keep='first', inplace=False)
    final_2018[i] = final_2018[i].drop_duplicates(subset=['訂單時間'])
    
    final_2018[i] = final_2018[i].reset_index(drop = True)
    
    del final_2018[i]['訂單時間']
    del final_2018[i]['系列']
    

In [18]:
final_2018

[           會員       廣告代號 第幾次購買    利潤
 0    10135347    1月_自然流量     1   956
 1    10135248     1月_KDP     1   498
 2       23524     1月_KDP     1   609
 3    10080582    1月_自然流量     1  1140
 4    10125335    1月_自然流量     1   512
 ..        ...        ...   ...   ...
 314     29996  1月_critei     1   564
 315     69118    1月_edmP     1   319
 316  10093209    1月_自然流量     1   469
 317  10076597  1月_critei     1   544
 318  10137781    1月_自然流量     1   748
 
 [319 rows x 4 columns],
            會員       廣告代號 第幾次購買    利潤
 0    10135347    2月_自然流量     2   722
 1    10134641    2月_自然流量     2  1308
 2    10090740  2月_critei     2   298
 3    10136036    2月_自然流量     2   583
 4    10136093  2月_critei     2   380
 ..        ...        ...   ...   ...
 311  10129017    2月_自然流量     1   446
 312  10140851     2月_KDP     1   704
 313  10140852    2月_自然流量     1   760
 314  10140862  2月_critei     1   161
 315  10124281   2月_GINEP     1   544
 
 [316 rows x 4 columns],
            會員       廣告代號 第幾次購買   

# 資料視覺化設計

In [21]:
for ad_num in range(len(ad_pd1)):
    
    ad_total=[]
    ad_total_mon = []
    ad_total_mon = pd.DataFrame(ad_total_mon)
        
    
    for i in range(len(final_2018)): 
        
        # 舉例：12月份 critei（廣告代號）的資料
        ad_mon = final_2018[i][final_2018[i]['廣告代號']==str(i+1)+'月_'+ad_pd1[ad_num]]
        # 重新排序index
        ad_mon = ad_mon.reset_index(drop = True)
        # 將月份資料加入ad_total中
        ad_total.append(ad_mon) 
            
            

        # 將單一個月的"第幾次購買"人數建立df    
        # 把每月份的資料以第幾次購買分群
        ad_mon_gp = ad_total[i].groupby('第幾次購買')
        # 求出每群利潤的總額
        ad_mon_profit = ad_total[i].groupby('第幾次購買')['利潤'].sum().tolist()
        # 建立DataFrame
        mon_final =pd.DataFrame({'月份':str(i+1)+'月',
                                '第幾次購買':ad_mon_gp.size().index.tolist(),
                                '人數':ad_mon_gp.size().values.tolist(),
                                '利潤':ad_mon_profit})
        ad_total_mon = pd.concat([ad_total_mon,mon_final])
        
        
        ad_total_mon['平均利潤'] = round(ad_total_mon['利潤'] / ad_total_mon['人數'] )
    

     
    # 將資料量太少的排除(可調整：目前為10)
    if len(ad_total_mon) < 10:
        continue
    
    
    # 計算每個月份利潤的總和
    mon_tt_profit = ad_total_mon.groupby('月份')['利潤'].sum()
    mon_tt_profit = mon_tt_profit.reset_index()
    
    
    # 計算每個月份購買人數
    mon_tt_people = ad_total_mon.groupby('月份')['人數'].sum()
    mon_tt_people = mon_tt_people.reset_index()
    
    
    # 計算月份人均利潤
    mon_tt_profit['平均利潤'] = round(mon_tt_profit['利潤'] / mon_tt_people['人數'])
        
    
    # 存取該廣告有利潤的月份
    mon_list = ad_total_mon['月份'].unique().tolist()
    
    
    # 先把 10 11 12月的資料存下來
    oct_nov_dec = []
    oct_nov_dec = pd.DataFrame(oct_nov_dec)
    for i in ['10月','11月','12月']:
        oct_nov_dec= pd.concat([oct_nov_dec,mon_tt_profit[mon_tt_profit['月份']== i]])

    
    # 把原本的順序在前面10 11 12月資料刪除
    if '10月' in mon_list:
        mon_tt_profit.drop(mon_tt_profit.loc[mon_tt_profit['月份']=='10月'].index, inplace=True)
    if '11月' in mon_list:
        mon_tt_profit.drop(mon_tt_profit.loc[mon_tt_profit['月份']=='11月'].index, inplace=True)
    if '12月' in mon_list:
        mon_tt_profit.drop(mon_tt_profit.loc[mon_tt_profit['月份']=='12月'].index, inplace=True)    
    # 在尾端補上 10 11 12月資料
    mon_tt_profit = pd.concat([mon_tt_profit,oct_nov_dec])
    mon_tt_profit = mon_tt_profit.reset_index(drop = True)
        
        
    
    # 年平均資料
    mon_mean_profit =[]
    for i in range(12):
        mon_mean_profit.append(round(mon_tt_profit.groupby('月份')['平均利潤'].sum().mean()))
    
    
    
    # 資料視覺化
    # 先畫bar + scatter(月平均) + scatter(lines型態,年平均)
    fig = px.bar(ad_total_mon,
                 x = '月份', y = '平均利潤',
                 color = '第幾次購買',barmode="group")
    
    # bar + scatter(月平均)
    fig.add_scatter(x = mon_list, y = mon_tt_profit.平均利潤, name = '月每人平均利潤')
    
    # bar + scatter(月平均) + scatter(lines型態,年平均)
    fig.add_scatter(x=mon_list,y=mon_mean_profit, name = '年每人平均利潤',mode='lines')
        
        
    
    
    
    # 調整外觀
    fig.update_layout(  
        title={'text': '系列1_'+ ad_pd1[ad_num] +'_利潤比較圖' ,
               'y':0.95,'x':0.2,
               'xanchor': 'center'},
        title_font_color="black",
        xaxis_title="月份",
        xaxis = dict(tickformat = ',d',
                     color = 'black'),
        yaxis_title="利潤",
        yaxis = dict(tickformat = ',d',
                     color = 'black'),
        autosize=False,
        width=1800,
        height=960,
        font=dict(family="Courier New, monospace",
                  size=20,
                  color="lightslategrey"),
        legend=dict(orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1),
    )
   
    pyplt = py.offline.plot 
    # 跑出圖檔
    pyplt(fig, filename =  ad_pd1[ad_num]+'_2018'+'.html',auto_open = False)

In [20]:
ad_total=[]
ad_total_mon = []
ad_total_mon = pd.DataFrame(ad_total_mon)


# 建立單一廣告每個月客群(第幾次購買)的資料
# 月份、第幾次購買、人數、利潤、每人平均利潤
for i in range(len(final_2018)): 


    # 彙整同一個廣告代號每個月的資料
    ad_mon = final_2018[i][final_2018[i]['廣告代號']==str(i+1)+'月_'+ad_pd1[1]]
    # 重新排序index
    ad_mon = ad_mon.reset_index(drop = True)
    # 將月份資料加入ad_total中
    ad_total.append(ad_mon)

    # 將單一個月的"第幾次購買"人數建立df    
    # 把每月份的資料以第幾次購買分群
    ad_mon_gp = ad_total[i].groupby('第幾次購買')
    # 求出每群利潤的總額
    ad_mon_profit = ad_total[i].groupby('第幾次購買')['利潤'].sum().tolist()
    # 建立DataFrame
    mon_final =pd.DataFrame({'月份':str(i+1)+'月',
                            '第幾次購買':ad_mon_gp.size().index.tolist(),
                            '人數':ad_mon_gp.size().values.tolist(),
                            '利潤':ad_mon_profit})
    ad_total_mon = pd.concat([ad_total_mon,mon_final])


    ad_total_mon['平均利潤'] = round(ad_total_mon['利潤'] / ad_total_mon['人數'] )


In [21]:
 ad_total_mon

,月份,第幾次購買,人數,利潤,平均利潤
0,1月,1,128,89026,696.0
1,1月,2,2,1475,738.0
0,2月,1,144,97316,676.0
1,2月,2,8,7106,888.0
0,3月,1,86,51616,600.0
1,3月,2,5,3353,671.0
0,4月,1,93,45587,490.0
1,4月,2,10,7263,726.0
0,5月,1,74,39161,529.0
1,5月,2,5,2551,510.0


In [25]:

    
ad_total=[]
ad_total_mon = []
ad_total_mon = pd.DataFrame(ad_total_mon)


# 建立單一廣告每個月客群(第幾次購買)的資料
# 月份、第幾次購買、人數、利潤、每人平均利潤
for i in range(len(final_2018)): 


    # 彙整同一個廣告代號每個月的資料
    ad_mon = final_2018[i][final_2018[i]['廣告代號']==str(i+1)+'月_'+ad_pd1[1]]
    # 重新排序index
    ad_mon = ad_mon.reset_index(drop = True)
    # 將月份資料加入ad_total中
    ad_total.append(ad_mon)

    # 將單一個月的"第幾次購買"人數建立df    
    # 把每月份的資料以第幾次購買分群
    ad_mon_gp = ad_total[i].groupby('第幾次購買')
    # 求出每群利潤的總額
    ad_mon_profit = ad_total[i].groupby('第幾次購買')['利潤'].sum().tolist()
    # 建立DataFrame
    mon_final =pd.DataFrame({'月份':str(i+1)+'月',
                            '第幾次購買':ad_mon_gp.size().index.tolist(),
                            '人數':ad_mon_gp.size().values.tolist(),
                            '利潤':ad_mon_profit})
    ad_total_mon = pd.concat([ad_total_mon,mon_final])


    ad_total_mon['平均利潤'] = round(ad_total_mon['利潤'] / ad_total_mon['人數'] )

# 計算每個月份利潤的總和
mon_tt_profit = ad_total_mon.groupby('月份')['利潤'].sum()
mon_tt_profit = mon_tt_profit.reset_index()

In [26]:
mon_tt_profit

,月份,利潤
0,10月,60032
1,11月,80951
2,12月,1396243
3,1月,90501
4,2月,104422
5,3月,54969
6,4月,52850
7,5月,42851
8,6月,118031
9,7月,209460


In [27]:

    
ad_total=[]
ad_total_mon = []
ad_total_mon = pd.DataFrame(ad_total_mon)


# 建立單一廣告每個月客群(第幾次購買)的資料
# 月份、第幾次購買、人數、利潤、每人平均利潤
for i in range(len(final_2018)): 


    # 彙整同一個廣告代號每個月的資料
    ad_mon = final_2018[i][final_2018[i]['廣告代號']==str(i+1)+'月_'+ad_pd1[1]]
    # 重新排序index
    ad_mon = ad_mon.reset_index(drop = True)
    # 將月份資料加入ad_total中
    ad_total.append(ad_mon)

    # 將單一個月的"第幾次購買"人數建立df    
    # 把每月份的資料以第幾次購買分群
    ad_mon_gp = ad_total[i].groupby('第幾次購買')
    # 求出每群利潤的總額
    ad_mon_profit = ad_total[i].groupby('第幾次購買')['利潤'].sum().tolist()
    # 建立DataFrame
    mon_final =pd.DataFrame({'月份':str(i+1)+'月',
                            '第幾次購買':ad_mon_gp.size().index.tolist(),
                            '人數':ad_mon_gp.size().values.tolist(),
                            '利潤':ad_mon_profit})
    ad_total_mon = pd.concat([ad_total_mon,mon_final])


    ad_total_mon['平均利潤'] = round(ad_total_mon['利潤'] / ad_total_mon['人數'] )

# 計算每個月份利潤的總和
mon_tt_profit = ad_total_mon.groupby('月份')['利潤'].sum()
mon_tt_profit = mon_tt_profit.reset_index()

# 計算每個月份購買人數
mon_tt_people = ad_total_mon.groupby('月份')['人數'].sum()
mon_tt_people = mon_tt_people.reset_index()


# 計算月份人均利潤
mon_tt_profit['平均利潤'] = round(mon_tt_profit['利潤'] / mon_tt_people['人數'])


# 存取該廣告有利潤的月份
mon_list = ad_total_mon['月份'].unique().tolist()

# 外部迴圈-程式碼 6
# 先把 10 11 12月的資料存下來
oct_nov_dec = []
oct_nov_dec = pd.DataFrame(oct_nov_dec)
for i in ['10月','11月','12月']:
    oct_nov_dec= pd.concat([oct_nov_dec,mon_tt_profit[mon_tt_profit['月份']== i]])

# 外部迴圈-程式碼 7
# 把原本的順序在前面10 11 12月資料刪除
if '10月' in mon_list:
    mon_tt_profit.drop(mon_tt_profit.loc[mon_tt_profit['月份']=='10月'].index, inplace=True)
if '11月' in mon_list:
    mon_tt_profit.drop(mon_tt_profit.loc[mon_tt_profit['月份']=='11月'].index, inplace=True)
if '12月' in mon_list:
    mon_tt_profit.drop(mon_tt_profit.loc[mon_tt_profit['月份']=='12月'].index, inplace=True)    
# 在尾端補上 10 11 12月資料
mon_tt_profit = pd.concat([mon_tt_profit,oct_nov_dec])
mon_tt_profit = mon_tt_profit.reset_index(drop = True)

In [28]:
mon_tt_profit

,月份,利潤,平均利潤
0,1月,90501,696.0
1,2月,104422,687.0
2,3月,54969,604.0
3,4月,52850,513.0
4,5月,42851,529.0
5,6月,118031,733.0
6,7月,209460,590.0
7,8月,126338,554.0
8,9月,91598,569.0
9,10月,60032,429.0


In [30]:

    
ad_total=[]
ad_total_mon = []
ad_total_mon = pd.DataFrame(ad_total_mon)


# 建立單一廣告每個月客群(第幾次購買)的資料
# 月份、第幾次購買、人數、利潤、每人平均利潤
for i in range(len(final_2018)): 


    # 彙整同一個廣告代號每個月的資料
    ad_mon = final_2018[i][final_2018[i]['廣告代號']==str(i+1)+'月_'+ad_pd1[1]]
    # 重新排序index
    ad_mon = ad_mon.reset_index(drop = True)
    # 將月份資料加入ad_total中
    ad_total.append(ad_mon)

    # 將單一個月的"第幾次購買"人數建立df    
    # 把每月份的資料以第幾次購買分群
    ad_mon_gp = ad_total[i].groupby('第幾次購買')
    # 求出每群利潤的總額
    ad_mon_profit = ad_total[i].groupby('第幾次購買')['利潤'].sum().tolist()
    # 建立DataFrame
    mon_final =pd.DataFrame({'月份':str(i+1)+'月',
                            '第幾次購買':ad_mon_gp.size().index.tolist(),
                            '人數':ad_mon_gp.size().values.tolist(),
                            '利潤':ad_mon_profit})
    ad_total_mon = pd.concat([ad_total_mon,mon_final])


    ad_total_mon['平均利潤'] = round(ad_total_mon['利潤'] / ad_total_mon['人數'] )

# 計算每個月份利潤的總和
mon_tt_profit = ad_total_mon.groupby('月份')['利潤'].sum()
mon_tt_profit = mon_tt_profit.reset_index()

# 計算每個月份購買人數
mon_tt_people = ad_total_mon.groupby('月份')['人數'].sum()
mon_tt_people = mon_tt_people.reset_index()


# 計算月份人均利潤
mon_tt_profit['平均利潤'] = round(mon_tt_profit['利潤'] / mon_tt_people['人數'])


# 存取該廣告有利潤的月份
mon_list = ad_total_mon['月份'].unique().tolist()

# 外部迴圈-程式碼 6
# 先把 10 11 12月的資料存下來
oct_nov_dec = []
oct_nov_dec = pd.DataFrame(oct_nov_dec)
for i in ['10月','11月','12月']:
    oct_nov_dec= pd.concat([oct_nov_dec,mon_tt_profit[mon_tt_profit['月份']== i]])

# 外部迴圈-程式碼 7
# 把原本的順序在前面10 11 12月資料刪除
if '10月' in mon_list:
    mon_tt_profit.drop(mon_tt_profit.loc[mon_tt_profit['月份']=='10月'].index, inplace=True)
if '11月' in mon_list:
    mon_tt_profit.drop(mon_tt_profit.loc[mon_tt_profit['月份']=='11月'].index, inplace=True)
if '12月' in mon_list:
    mon_tt_profit.drop(mon_tt_profit.loc[mon_tt_profit['月份']=='12月'].index, inplace=True)    
# 在尾端補上 10 11 12月資料
mon_tt_profit = pd.concat([mon_tt_profit,oct_nov_dec])
mon_tt_profit = mon_tt_profit.reset_index(drop = True)

# 年平均資料
mon_mean_profit =[]
for i in range(12):
    mon_mean_profit.append(round(mon_tt_profit.groupby('月份')['平均利潤'].sum().mean()))




In [32]:
mon_tt_profit 

,月份,利潤,平均利潤
0,1月,90501,696.0
1,2月,104422,687.0
2,3月,54969,604.0
3,4月,52850,513.0
4,5月,42851,529.0
5,6月,118031,733.0
6,7月,209460,590.0
7,8月,126338,554.0
8,9月,91598,569.0
9,10月,60032,429.0


In [49]:

    
ad_total=[]
ad_total_mon = []
ad_total_mon = pd.DataFrame(ad_total_mon)


# 建立單一廣告每個月客群(第幾次購買)的資料
# 月份、第幾次購買、人數、利潤、每人平均利潤
for i in range(len(final_2018)): 


    # 彙整同一個廣告代號每個月的資料
    ad_mon = final_2018[i][final_2018[i]['廣告代號']==str(i+1)+'月_'+'critei']
    # 重新排序index
    ad_mon = ad_mon.reset_index(drop = True)
    # 將月份資料加入ad_total中
    ad_total.append(ad_mon)

    # 將單一個月的"第幾次購買"人數建立df    
    # 把每月份的資料以第幾次購買分群
    ad_mon_gp = ad_total[i].groupby('第幾次購買')
    # 求出每群利潤的總額
    ad_mon_profit = ad_total[i].groupby('第幾次購買')['利潤'].sum().tolist()
    # 建立DataFrame
    mon_final =pd.DataFrame({'月份':str(i+1)+'月',
                            '第幾次購買':ad_mon_gp.size().index.tolist(),
                            '人數':ad_mon_gp.size().values.tolist(),
                            '利潤':ad_mon_profit})
    ad_total_mon = pd.concat([ad_total_mon,mon_final])


    ad_total_mon['平均利潤'] = round(ad_total_mon['利潤'] / ad_total_mon['人數'] )

# 計算每個月份利潤的總和
mon_tt_profit = ad_total_mon.groupby('月份')['利潤'].sum()
mon_tt_profit = mon_tt_profit.reset_index()

# 計算每個月份購買人數
mon_tt_people = ad_total_mon.groupby('月份')['人數'].sum()
mon_tt_people = mon_tt_people.reset_index()


# 計算月份人均利潤
mon_tt_profit['平均利潤'] = round(mon_tt_profit['利潤'] / mon_tt_people['人數'])


# 存取該廣告有利潤的月份
mon_list = ad_total_mon['月份'].unique().tolist()

# 外部迴圈-程式碼 6
# 先把 10 11 12月的資料存下來
oct_nov_dec = []
oct_nov_dec = pd.DataFrame(oct_nov_dec)
for i in ['10月','11月','12月']:
    oct_nov_dec= pd.concat([oct_nov_dec,mon_tt_profit[mon_tt_profit['月份']== i]])

# 外部迴圈-程式碼 7
# 把原本的順序在前面10 11 12月資料刪除
if '10月' in mon_list:
    mon_tt_profit.drop(mon_tt_profit.loc[mon_tt_profit['月份']=='10月'].index, inplace=True)
if '11月' in mon_list:
    mon_tt_profit.drop(mon_tt_profit.loc[mon_tt_profit['月份']=='11月'].index, inplace=True)
if '12月' in mon_list:
    mon_tt_profit.drop(mon_tt_profit.loc[mon_tt_profit['月份']=='12月'].index, inplace=True)    
# 在尾端補上 10 11 12月資料
mon_tt_profit = pd.concat([mon_tt_profit,oct_nov_dec])
mon_tt_profit = mon_tt_profit.reset_index(drop = True)

# 年平均資料
mon_mean_profit =[]
for i in range(12):
    mon_mean_profit.append(round(mon_tt_profit.groupby('月份')['平均利潤'].sum().mean()))

# 先畫bar + scatter(月平均) + scatter(lines型態,年平均)
fig = px.bar(ad_total_mon,
             x = '月份', y = '平均利潤',
             color = '第幾次購買',barmode="group")


ValueError: cannot convert float NaN to integer

In [41]:
fig

In [38]:
ad_total=[]
ad_total_mon = []
ad_total_mon = pd.DataFrame(ad_total_mon)


# 建立單一廣告每個月客群(第幾次購買)的資料
# 月份、第幾次購買、人數、利潤、每人平均利潤
for i in range(len(final_2018)): 


    # 彙整同一個廣告代號每個月的資料
    ad_mon = final_2018[i][final_2018[i]['廣告代號']==str(i+1)+'月_'+ad_pd1[1]]
    # 重新排序index
    ad_mon = ad_mon.reset_index(drop = True)
    # 將月份資料加入ad_total中
    ad_total.append(ad_mon)

    # 將單一個月的"第幾次購買"人數建立df    
    # 把每月份的資料以第幾次購買分群
    ad_mon_gp = ad_total[i].groupby('第幾次購買')
    # 求出每群利潤的總額
    ad_mon_profit = ad_total[i].groupby('第幾次購買')['利潤'].sum().tolist()
    # 建立DataFrame
    mon_final =pd.DataFrame({'月份':str(i+1)+'月',
                            '第幾次購買':ad_mon_gp.size().index.tolist(),
                            '人數':ad_mon_gp.size().values.tolist(),
                            '利潤':ad_mon_profit})
    ad_total_mon = pd.concat([ad_total_mon,mon_final])


    ad_total_mon['平均利潤'] = round(ad_total_mon['利潤'] / ad_total_mon['人數'] )

# 計算每個月份利潤的總和
mon_tt_profit = ad_total_mon.groupby('月份')['利潤'].sum()
mon_tt_profit = mon_tt_profit.reset_index()

# 計算每個月份購買人數
mon_tt_people = ad_total_mon.groupby('月份')['人數'].sum()
mon_tt_people = mon_tt_people.reset_index()


# 計算月份人均利潤
mon_tt_profit['平均利潤'] = round(mon_tt_profit['利潤'] / mon_tt_people['人數'])


# 存取該廣告有利潤的月份
mon_list = ad_total_mon['月份'].unique().tolist()

# 外部迴圈-程式碼 6
# 先把 10 11 12月的資料存下來
oct_nov_dec = []
oct_nov_dec = pd.DataFrame(oct_nov_dec)
for i in ['10月','11月','12月']:
    oct_nov_dec= pd.concat([oct_nov_dec,mon_tt_profit[mon_tt_profit['月份']== i]])

# 外部迴圈-程式碼 7
# 把原本的順序在前面10 11 12月資料刪除
if '10月' in mon_list:
    mon_tt_profit.drop(mon_tt_profit.loc[mon_tt_profit['月份']=='10月'].index, inplace=True)
if '11月' in mon_list:
    mon_tt_profit.drop(mon_tt_profit.loc[mon_tt_profit['月份']=='11月'].index, inplace=True)
if '12月' in mon_list:
    mon_tt_profit.drop(mon_tt_profit.loc[mon_tt_profit['月份']=='12月'].index, inplace=True)    
# 在尾端補上 10 11 12月資料
mon_tt_profit = pd.concat([mon_tt_profit,oct_nov_dec])
mon_tt_profit = mon_tt_profit.reset_index(drop = True)

# 年平均資料
mon_mean_profit =[]
for i in range(12):
    mon_mean_profit.append(round(mon_tt_profit.groupby('月份')['平均利潤'].sum().mean()))

# 先畫bar + scatter(月平均) + scatter(lines型態,年平均)
fig = px.bar(ad_total_mon,
             x = '月份', y = '平均利潤',
             color = '第幾次購買',barmode="group")
# bar + scatter(月平均)
fig.add_scatter(x = mon_list, y = mon_tt_profit.平均利潤, name = '月每人平均利潤')
# 資料視覺化-程式碼 3
# bar + scatter(月平均) + scatter(lines型態,年平均)
fig.add_scatter(x=mon_list,y=mon_mean_profit, name = '年每人平均利潤',mode='lines')



In [39]:
# 調整外觀
fig.update_layout(  
    title={'text': '系列1_'+ ad_pd1[ad_num] +'_利潤比較圖' ,
           'y':0.95,'x':0.2,
           'xanchor': 'center'},
    title_font_color="black",
    xaxis_title="月份",
    xaxis = dict(tickformat = ',d',
                 color = 'black'),
    yaxis_title="利潤",
    yaxis = dict(tickformat = ',d',
                 color = 'black'),
    autosize=False,
    width=1800,
    height=960,
    font=dict(family="Courier New, monospace",
              size=20,
              color="lightslategrey"),
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
)

In [16]:
for ad_num in range(len(ad_pd1)):
    
    ad_total=[]
    ad_total_mon = []
    ad_total_mon = pd.DataFrame(ad_total_mon)
        
    
    # 建立單一廣告每個月客群(第幾次購買)的資料
    # 月份、第幾次購買、人數、利潤、每人平均利潤
    for i in range(len(final_2018)): 
        
        
        # 彙整同一個廣告代號每個月的資料
        ad_mon = final_2018[i][final_2018[i]['廣告代號']==str(i+1)+'月_'+ad_pd1[ad_num]]
        # 重新排序index
        ad_mon = ad_mon.reset_index(drop = True)
        # 將月份資料加入ad_total中
        ad_total.append(ad_mon)
       
        # 將單一個月的"第幾次購買"人數建立df    
        # 把每月份的資料以第幾次購買分群
        ad_mon_gp = ad_total[i].groupby('第幾次購買')
        # 求出每群利潤的總額
        ad_mon_profit = ad_total[i].groupby('第幾次購買')['利潤'].sum().tolist()
        # 建立DataFrame
        mon_final =pd.DataFrame({'月份':str(i+1)+'月',
                                '第幾次購買':ad_mon_gp.size().index.tolist(),
                                '人數':ad_mon_gp.size().values.tolist(),
                                '利潤':ad_mon_profit})
        ad_total_mon = pd.concat([ad_total_mon,mon_final])
        
        
        ad_total_mon['平均利潤'] = round(ad_total_mon['利潤'] / ad_total_mon['人數'] )
    

    
    # 將資料量太少的排除(可調整：目前為10)
    #if len(ad_total_mon) < 10:
    #    continue
    
   
    # 計算每個月份利潤的總和
    mon_tt_profit = ad_total_mon.groupby('月份')['利潤'].sum()
    mon_tt_profit = mon_tt_profit.reset_index()
    
    
    # 計算每個月份購買人數
    mon_tt_people = ad_total_mon.groupby('月份')['人數'].sum()
    mon_tt_people = mon_tt_people.reset_index()
    
   
    # 計算月份人均利潤
    mon_tt_profit['平均利潤'] = round(mon_tt_profit['利潤'] / mon_tt_people['人數'])
        
   
    # 存取該廣告有利潤的月份
    mon_list = ad_total_mon['月份'].unique().tolist()
    
    # 外部迴圈-程式碼 6
    # 先把 10 11 12月的資料存下來
    oct_nov_dec = []
    oct_nov_dec = pd.DataFrame(oct_nov_dec)
    for i in ['10月','11月','12月']:
        oct_nov_dec= pd.concat([oct_nov_dec,mon_tt_profit[mon_tt_profit['月份']== i]])

    # 外部迴圈-程式碼 7
    # 把原本的順序在前面10 11 12月資料刪除
    if '10月' in mon_list:
        mon_tt_profit.drop(mon_tt_profit.loc[mon_tt_profit['月份']=='10月'].index, inplace=True)
    if '11月' in mon_list:
        mon_tt_profit.drop(mon_tt_profit.loc[mon_tt_profit['月份']=='11月'].index, inplace=True)
    if '12月' in mon_list:
        mon_tt_profit.drop(mon_tt_profit.loc[mon_tt_profit['月份']=='12月'].index, inplace=True)    
    # 在尾端補上 10 11 12月資料
    mon_tt_profit = pd.concat([mon_tt_profit,oct_nov_dec])
    mon_tt_profit = mon_tt_profit.reset_index(drop = True)
        
        
    
    # 年平均資料
    mon_mean_profit =[]
    for i in range(12):
        mon_mean_profit.append(round(mon_tt_profit.groupby('月份')['平均利潤'].sum().mean()))
    
    # 資料視覺化

    # 先畫bar + scatter(月平均) + scatter(lines型態,年平均)
    fig = px.bar(ad_total_mon,
                 x = '月份', y = '平均利潤',
                 color = '第幾次購買',barmode="group")
    
    # bar + scatter(月平均)
    fig.add_scatter(x = mon_list, y = mon_tt_profit.平均利潤, name = '月每人平均利潤')
    
    # bar + scatter(月平均) + scatter(lines型態,年平均)
    fig.add_scatter(x=mon_list,y=mon_mean_profit, name = '年每人平均利潤',mode='lines')
        
        
    
    
    
    # 調整外觀
    fig.update_layout(  
        title={'text': '系列1_'+ ad_pd1[ad_num] +'_利潤比較圖' ,
               'y':0.95,'x':0.2,
               'xanchor': 'center'},
        title_font_color="black",
        xaxis_title="月份",
        xaxis = dict(tickformat = ',d',
                     color = 'black'),
        yaxis_title="利潤",
        yaxis = dict(tickformat = ',d',
                     color = 'black'),
        autosize=False,
        width=1800,
        height=960,
        font=dict(family="Courier New, monospace",
                  size=20,
                  color="lightslategrey"),
        legend=dict(orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1),
    )
   
    

ValueError: cannot convert float NaN to integer

In [ ]:
pyplt = py.offline.plot 
# 跑出圖檔
pyplt(fig, filename =  ad_pd1[ad_num]+'_2018'+'.html',auto_open = False)